## Window functions

In [1]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("Window functions").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

In [4]:
spark